In [69]:
print("Ok!")

Ok!


In [50]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [6]:
pip install pypdf

In [36]:
!pip install -qU \
  pinecone-client==3.1.0 

In [37]:
PINECONE_API_KEY="29308536-db0f-4eba-ae12-eb031acd2561"
PINECONE_API_ENV="us-east1-aws"

In [38]:
#Extract Data from the PDF
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
   
    documents=loader.load()

    return documents

In [39]:
extracted_data=load_pdf("data/")

In [40]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [41]:
text_chunks=text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 6830


In [9]:
#text_chunks

In [42]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [43]:
embeddings= download_hugging_face_embeddings()

In [44]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [45]:
query_result= embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [184]:
#query_result

In [185]:
#text_chunks

In [52]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = '29308536-db0f-4eba-ae12-eb031acd2561'

# configure client
pc = Pinecone(api_key=api_key)

In [53]:
pc

In [54]:
from pinecone import ServerlessSpec

cloud =  'aws'
region =  'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [55]:
index_name = "schatbot"
index = pc.Index(index_name)
index

In [58]:
def docsearch():
    for i, t in zip(range(len(text_chunks)), text_chunks):
        query_result = embeddings.embed_query(t.page_content)
        index.upsert(
            vectors=[
                {
                    "id": str(i),  # Convert i to a string
                    "values": query_result, 
                    "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
                }
             ],
             namespace="real" 
        )

 

In [ ]:
docsearch=docsearch()

In [ ]:
index.describe_index_stats()

In [125]:
#docs_chunks=[t.page_content for t in text_chunks]
#docsearch=Pinecone.from_texts(docs_chunks, embeddings, index_name='schatbot')

AttributeError: from_texts is not a top-level attribute of the Pinecone class provided by pinecone's official python package developed at https://github.com/pinecone-io/pinecone-python-client. You may have a name collision with an export from another dependency in your project that wraps Pinecone functionality and exports a similarly named class. Please refer to the following knowledge base article for more information: https://docs.pinecone.io/troubleshooting/pinecone-attribute-errors-with-langchain


In [106]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don'try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [107]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [119]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type='llama',
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [120]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retreiver=.as_retreiver(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

MaxRetryError: HTTPSConnectionPool(host='schatbot-68wd8r8.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /vectors/upsert (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000253C7E51850>: Failed to resolve 'schatbot-68wd8r8.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))